In [ ]:
!uv add --active --dev openai python-dotenv tavily-python

In [18]:
import os
from openai import OpenAI
from dotenv import load_dotenv
def load_openai_client() -> OpenAI:
    try:
        load_dotenv(dotenv_path=".env")
    except ImportError:
        print("💡 Tip: Install python-dotenv with `pip install python-dotenv`")

    api_key = os.getenv("OPENAI_API_KEY")

    if not api_key:
        raise ValueError("❌ OPENAI_API_KEY not found. Set it in .env or via os.environ.")

    return OpenAI(api_key=api_key)

In [19]:
import os
import json
from typing import Callable, Dict
from openai import OpenAI
from tavily import TavilyClient
import os
from dotenv import load_dotenv

# Setup OpenAI client
client = load_openai_client()

def search_tool(query: str, max_results: int = 3) -> str:
    api_key = os.getenv("TAVILY_API_KEY")
    if not api_key:
        raise ValueError("TAVILY_API_KEY not found in environment.")

    client = TavilyClient(api_key=api_key)
    response = client.search(query=query, max_results=max_results)

    if not response or "results" not in response:
        return "No results found."

    return response["results"]

def calculator_tool(expr: str) -> str:
    try:
        return str(eval(expr))
    except Exception as e:
        return f"Error: {e}"

TOOLS: Dict[str, Callable[[str], str]] = {
    "search": search_tool,
    "calculate": calculator_tool,
}


In [20]:
search_tool("what is Rust?")

[{'url': 'https://en.wikipedia.org/wiki/Rust',
  'title': 'Rust - Wikipedia',
  'content': 'Rust is an iron oxide, a usually reddish-brown oxide formed by the reaction of iron and oxygen in the catalytic presence of water or air moisture.',
  'score': 0.918838,
  'raw_content': None},
 {'url': 'https://www.metaltek.com/blog/what-is-rust-and-how-to-prevent-it/',
  'title': 'What Is Rust And How To Prevent It - MetalTek International',
  'content': 'Rust is a form of corrosion that primarily affects iron and its alloys, such as steel. The chemical reaction between iron, oxygen, and water forms iron oxide,',
  'score': 0.8358192,
  'raw_content': None},
 {'url': 'https://www.rust-lang.org/',
  'title': 'Rust Programming Language',
  'content': 'Rust Programming Language Rust Logo # Rust ## Why Rust? Rust has great documentation, a friendly compiler with useful error ## Build it in Rust Rust is great for network services. Rust has you covered. ## Rust in production corporations, from embed

In [21]:
def run_agent_notebook(query: str, max_iterations: int = 3):
    system_prompt = """
You are a reasoning agent. Your job is to answer multi-part user questions by first planning tool usage.

You can use the following tools:
- "search" → for looking up concepts (e.g., "What is Python?")
- "calculate" → for numeric math (e.g., "2 * 10")

Your response must always be JSON:
{
  "tool": "search" | "calculate" | null,
  "input": string,
  "final_answer": optional string
}

Only include "final_answer" if you have answered **all parts** of the user question using available tools.
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]

    for i in range(max_iterations):
        print(f"\n🧠 Iteration {i+1}")

        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0
        )

        content = response.choices[0].message.content
        print("📩 LLM:", content)

        try:
            parsed = json.loads(content)
        except json.JSONDecodeError as e:
            print("❌ JSON parsing failed:", e)
            break

        if parsed.get("final_answer"):
            print(f"\n✅ Final Answer: {parsed['final_answer']}")
            break

        tool = parsed.get("tool")
        tool_input = parsed.get("input")

        if tool not in TOOLS:
            print(f"❌ Unknown tool: {tool}")
            break

        output = TOOLS[tool](tool_input)
        print(f"🛠️ `{tool}` → {output}")

        # Feed back into LLM
        messages.append({"role": "assistant", "content": content})
        messages.append({"role": "user", "content": f"Tool `{tool}` output: {output}"})


In [22]:
run_agent_notebook("What is Java and what is 2 * 10?")


🧠 Iteration 1
📩 LLM: {
  "tool": "search",
  "input": "What is Java"
}
🛠️ `search` → [{'url': 'https://www.geeksforgeeks.org/java/introduction-to-java/', 'title': 'Introduction to Java - GeeksforGeeks', 'content': 'Java is a high-level, object-oriented programming language developed by Sun Microsystems in 1995. It is platform-independent, which means we can', 'score': 0.8497846, 'raw_content': None}, {'url': 'https://aws.amazon.com/what-is/java/', 'title': 'What is Java? - Java Programming Language Explained - AWS', 'content': '# What is Java? ## What is Java? Java is a widely-used programming language for coding web applications. ## What is Java programming language used for? ## What is Java programming language used for? Cloud providers choose Java language to run programs on a wide range of underlying platforms. Some reasons developers continue to choose Java over other programming languages include: This is the front-end communication between the developer and the Java platform. T

In [106]:
import ollama
import json
import re

def format_tavily_results(results: list[dict], max_entries: int = 3) -> str:
    if not results:
        return "No relevant information found."

    lines = []
    for i, item in enumerate(results[:max_entries]):
        title = item.get("title", "Untitled")
        url = item.get("url", "")
        content = item.get("content", "") or item.get("raw_content", "")

        # Clean content: remove line breaks, truncate if too long
        clean_content = content.replace("\n", " ").strip()
        if len(clean_content) > 500:
            clean_content = clean_content[:500].rsplit(" ", 1)[0] + "..."

        lines.append(f"📄 **{title}**\n🔗 {url}\n📝 {clean_content}")

    return "\n\n".join(lines)
    
def run_ollama_agent(query: str, model: str = "llama3", max_iterations: int = 4):
    system_prompt = """
You are a reasoning agent. Your job is to answer multi-part user questions by first planning tool usage.

You can use the following tools:
- "search" → for looking up concepts (e.g., "What is Python?")
- "calculate" → for numeric math (e.g., "2 * 10")

If a tool has already been called with the same input, do not call it again. Use the result already provided in the conversation to answer the question.
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    search_tool_schema = {
    "type": "function",
    "function": {
        "name": "search",
        "description": "Search the web for current and relevant information.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Search query to look up",
                },
            },
            "required": ["query"],
        },
    },
}
    available_tools: Dict[str, Callable] = {
    'search': search_tool,}
    
    for i in range(max_iterations):
        print(f"\n🧠 Iteration {i+1}")

        response = ollama.chat(
            model=model,
            messages=messages,
            tools=[search_tool_schema],
        )

        content = response['message'].get('content', '').strip()
        if content:
           print("📩 LLM:", content)
        else:
           print("📩 LLM: [no message content — tool call or continuation]")


        if response.message.tool_calls:
            for tool in response.message.tool_calls or []:
                if function_to_call := available_tools.get(tool.function.name):
                    print('Calling function:', tool.function.name)
                    print('Arguments:', tool.function.arguments)
                    tool_input = tool.function.arguments
                    output = format_tavily_results(function_to_call(**tool.function.arguments))
                    print('Function output:', output)
                    messages.append({"role": "assistant", "content": f"I used the `search` tool with query: '{tool_input}' and got:\n{output}"})
                else:
                    print('Function', tool.function.name, 'not found')
        else:
            break
    

    

In [108]:
run_ollama_agent("What is the weather in Athens? 2*6666=?", model="llama3.1:8b")


🧠 Iteration 1
📩 LLM: [no message content — tool call or continuation]
Calling function: search
Arguments: {'query': 'athens weather'}
Function output: 📄 **Weather in Athens, Greece**
🔗 https://www.weatherapi.com/
📝 {'location': {'name': 'Athens', 'region': 'Attica', 'country': 'Greece', 'lat': 37.9795, 'lon': 23.7162, 'tz_id': 'Europe/Athens', 'localtime_epoch': 1753278192, 'localtime': '2025-07-23 16:43'}, 'current': {'last_updated_epoch': 1753277400, 'last_updated': '2025-07-23 16:30', 'temp_c': 38.2, 'temp_f': 100.8, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 194, 'wind_dir':...

📄 **Athens weather in July 2025 - Weather25.com**
🔗 https://www.weather25.com/europe/greece/attiki/athens?page=month&month=July
📝 Athens weather in July 2025 | Athens 14 day weather Athens Image 3: weather in Greece 5.    Athens Athens weather in July 2025 The weather in Athens in Jul